# GridWorld

In some cases the agent tries to move outside the grid and the cell that receives the negative reward is the origin of the agent's transition.
In other cases the agent moves into a high reward cell (like A or B) and the cell that receives the positive reward is the destimation of the agent's transition (A or B)

Implementation notes:
   - The state is the agent position on the grid.
   - The action is the movement of the agent in one time unit.

In [16]:
import numpy as np
from collections import defaultdict

In [99]:
class Agent:
    def __init__(self, x, y):
        self.state = (x, y)
        self.pi = [.25, .25, .25, .25]
        self.actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
                   
        # counters
        self.accum_rewards = defaultdict(int)
        self.cell_freqs = defaultdict(int)
    
    def choose_action(self):
        i = np.random.choice(range(len(self.actions)), p=self.pi)
        return self.actions[i], self.pi[i]
    
    def update_credit(self, s_temp, stplus1, r):
        # if it didin't move is because it jumped out of the grid
        if stplus1 == self.state:
            credit_cell = self.state
        else:
            credit_cell = s_temp            
            
        self.cell_freqs[credit_cell] += 1
        self.accum_rewards[credit_cell] += r 
        
        return credit_cell
    
    def update_state(self, stplus1):   
        self.state = stplus1

In [169]:
class Gridworld:
    def __init__(self, n):
        self.n = n
        
        # (i, j), 0,0) is the top-left corner of the grid
        self.A = [(0, 1), (4, 1), 10]
        self.B = [(0, 3), (2, 3), 5]
    
    def compute_reward(self, s_temp):
        r = 0
        
        if s_temp == self.A[0]:
            r = self.A[2]         
        elif s_temp == self.B[0]:
            r = self.B[2]          
        elif s_temp[0]<0 or s_temp[0]>self.n-1:
            r = -1           
        elif s_temp[1]<0 or s_temp[1]>self.n-1:
            r = -1
        
        return r
    
    
    def next_state(self, st, a):
        
        s_temp = (st[0] + a[0], st[1] + a[1])  # element-wise addition of tuples
        stplus1 = s_temp
        
        if stplus1 == self.A[0]:
            stplus1 = self.B[1]
            
        elif stplus1 == self.B[0]:
            stplus1 = self.B[1]
            
        elif stplus1[0]<0 or stplus1[0]>self.n-1:
            stplus1 = st
            
        elif stplus1[1]<0 or stplus1[1]>self.n-1:
            stplus1 = st
        
        return s_temp, stplus1


    def run(self, agent, num_steps):
        
        for _ in range(num_steps):
            a, p = agent.choose_action()
            s_temp, stplus1 = self.next_state(agent.state, a)   
            r = self.compute_reward(s_temp)
            agent.update_credit(s_temp, stplus1, r)
            agent.update_state(stplus1)

            
    def state_value_random_policy(self, agent, discount=0.9, error=1e-5):
        grid = np.zeros((n, n))

        for c in range(1000):
            grid_new = np.zeros((n, n))
            # equation (3.12):
            # (i, j) covers all possible states (aka s')
            for i in range(n):
                for j in range(n):
                    st = (i, j)
                    if st == self.A[0]:
                        st = self.A[1]
                        r = self.A[2]
                        grid_new[i, j] += 0.25 * (r + discount * grid[st[0], st[1]])
                    elif st == self.B[0]:
                        st = self.B[1]
                        r = self.B[2]
                        grid_new[i, j] += 0.25 * (r + discount * grid[st[0], st[1]])
                    else:
                        # average for all actions 
                        for a in agent.actions:
                            r = 0
                            stplus1 = (st[0] + a[0], st[1] + a[1])  # element-wise addition of tuples
                            if stplus1[0]<0 or stplus1[0]>self.n-1 or stplus1[1]<0 or stplus1[1]>self.n-1:
                                stplus1 = st
                                r = -1
                            #print st, a, stplus1, r
                            grid_new[i, j] += 0.25 * (r + discount * grid[stplus1[0], stplus1[1]])
                            #print grid_new[i, j], r, discount, grid[stplus1[0], stplus1[1]]

            if np.sum(np.abs(grid - grid_new)) < error: break
            grid = grid_new

        return np.round(grid, decimals=2)

           
    def state_value_optimal_policy(self, agent, discount=0.99, error=1e-5):
        grid = np.zeros((n, n))

        while True:
            grid_new = np.zeros((n, n))
            # (3.17): Bellman's optimality equation
            # (i, j) covers all possible states (aka s')
            for i in range(n):
                for j in range(n):
                    st = (i, j)
                    if st == self.A[0]:
                        st = self.A[1]
                        r = self.A[2]
                        grid_new[i, j] = 0.25 * (r + discount * grid[st[0], st[1]])
                    elif st == self.B[0]:
                        st = self.B[1]
                        r = self.B[2]
                        grid_new[i, j] = 0.25 * (r + discount * grid[st[0], st[1]])
                    else:
                        # maximize for all actions
                        action_values = []
                        for a in agent.actions:
                            r = 0
                            stplus1 = (st[0] + a[0], st[1] + a[1])  # element-wise addition of tuples
                            if stplus1[0]<0 or stplus1[0]>self.n-1 or stplus1[1]<0 or stplus1[1]>self.n-1:
                                stplus1 = st
                                r = -1
                            action_values.append(0.25 * (r + discount * grid[stplus1[0], stplus1[1]]))
                        grid_new[i, j] = max(action_values)
            if np.sum(np.abs(grid - grid_new)) < error: break
            grid = grid_new

        return np.round(grid, decimals=2)

# Simulation

In [170]:
n = 5
steps = 10000

grid_world = Gridworld(n)
agent = Agent(2, 2)
grid_world.run(agent, steps)

In [171]:
r = np.zeros(shape=(n, n))
f = np.zeros(shape=(n, n))
g = np.zeros(shape=(n, n))
for s in agent.cell_freqs.keys():
    r[s] = agent.accum_rewards[s] 
    f[s] = agent.cell_freqs[s]
    g[s] = agent.accum_rewards[s] / float(max(1, agent.cell_freqs[s]))

print np.round(g, decimals=2)


[[ -0.45  10.    -0.24   5.    -0.42]
 [ -0.29   0.     0.     0.    -0.25]
 [ -0.24   0.     0.     0.    -0.25]
 [ -0.25   0.     0.     0.    -0.28]
 [ -0.53  -0.25  -0.25  -0.26  -0.48]]


# State-value function for the equiprobable random policy

This value function was computed by solving the system of linear equations (3.12), for the discounted
reward case with γ = 0.9.                                         

In [172]:
print grid_world.state_value_random_policy(agent)

[[-0.35  2.09  0.61  1.09 -0.87]
 [-0.72  0.18  0.03 -0.14 -1.  ]
 [-1.2  -0.61 -0.53 -0.72 -1.33]
 [-1.69 -1.16 -1.03 -1.21 -1.75]
 [-2.35 -1.83 -1.69 -1.86 -2.39]]


# State-value function for the Optimal policy

In [173]:
print grid_world.state_value_optimal_policy(agent)

[[ 0.62  2.5   0.62  1.27  0.31]
 [ 0.15  0.62  0.15  0.31  0.08]
 [ 0.04  0.15  0.04  0.08  0.02]
 [ 0.01  0.04  0.01  0.02  0.  ]
 [ 0.    0.01  0.    0.    0.  ]]
